In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

import matplotlib.pyplot as plt
import random
import zipfile
import pandas as pd
%matplotlib inline


f = zipfile.ZipFile("./drive/My Drive/AOI/train_images.zip")
f.extractall("./datasets")

df = pd.read_csv("./drive/My Drive/AOI/train.csv")

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization

cnn = VGG16(include_top=False, input_shape=(256, 256, 3))

for l in cnn.layers:
    l.trainable = False

x = BatchNormalization()(cnn.output)
x = Flatten()(x)
x = Dense(64, activation="relu")(x)
out = Dense(6, activation="softmax")(x)

model2 = Model(inputs=cnn.input, outputs=out)
model2.summary()

In [ ]:
# 優化器 - Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model2.compile(loss=SparseCategoricalCrossentropy(),
               optimizer="adam",
               metrics=["accuracy"])

In [8]:
# 切割 train and test
# 要用 np.array 才不容易出錯

import numpy as np
from sklearn.model_selection import train_test_split

imagePaths = np.array("./datasets/train_images/" + df["ID"])
label = np.array(df["Label"])

# 圖片, 編號, 圖片, 編號
x_train, x_test, y_train, y_test = train_test_split(imagePaths,
                                                    label,
                                                    test_size=0.2)

[0 1 1 ... 3 0 5]
(2528,)

['./datasets/train_images/train_00000.png'
 './datasets/train_images/train_00001.png'
 './datasets/train_images/train_00002.png' ...
 './datasets/train_images/train_02525.png'
 './datasets/train_images/train_02526.png'
 './datasets/train_images/train_02527.png']


In [ ]:
dagen = ImageDataGenerator(rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               channel_shift_range=10,
                               horizontal_flip=True,
                               fill_mode='nearest')

In [93]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img

def data_generator(data_paths, batch_size):
    n = len(data_paths)
    i = 0
    data_paths = data_paths
    while True:
        image_data = []
        class_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(data_paths)
            path = data_paths[i]
            
            img = load_img(path, target_size=(256, 256))
            img_pre = preprocess_input(np.array(img))
            image_data.append(img_pre)
            i = (i+1) % n
        image_data = np.array(image_data)
        yield image_data

In [ ]:
EPOCHS = 10
BATCHS = 32

for i in range(epochs):
    batches = 0
    for x_batch, y_batch in dagen.flow(data_generator(x_train), data_generator(y_train), 32):
        model.fit(x_batch, y_batch)
        batches += 1
        if batches >= len(x_train) / BATCHS:
            break

In [52]:
# 拿 10% 的 200 張 算 evaluate = ['loss', 'acc']
timg, tx, ty = dataflow(x_test, y_test, 200)
model2.evaluate(tx, ty)

# 拿 10% 的驗證資料看機率值
pred = model2.predict(x_prepro02)
# pred

NameError: name 'x_test' is not defined

In [ ]:
# 載入 真．驗證資料

import zipfile
f = zipfile.ZipFile("./drive/My Drive/AOI/test_images.zip")
f.extractall("./datasets_test/")

In [14]:
# 讀 真．驗證資料 的答案

import pandas as pd

df2 = pd.read_csv("./drive/My Drive/AOI/test.csv")
df2

,ID,Label
0,test_00000.png,NaN
1,test_00001.png,NaN
2,test_00002.png,NaN
3,test_00003.png,NaN
4,test_00004.png,NaN
...,...,...
10137,test_10137.png,NaN
10138,test_10138.png,NaN
10139,test_10139.png,NaN
10140,test_10140.png,NaN


In [15]:
# 來預言喔

# 1: 讀圖片

import glob
path = glob.glob("datasets_test/test_images/*")
# print(path)

# 按名稱排序
path = sorted(path)

# 2: 路徑轉成np.array
npPath = np.array(path)
print(npPath)

['datasets_test/test_images/test_00000.png'
 'datasets_test/test_images/test_00001.png'
 'datasets_test/test_images/test_00002.png' ...
 'datasets_test/test_images/test_10139.png'
 'datasets_test/test_images/test_10140.png'
 'datasets_test/test_images/test_10141.png']


In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img

def dataflow2(npPath):

  AA = []

  for i, fp in enumerate(npPath):
    img = load_img(fp, target_size=(512, 512))

    img_np = np.array(img)

    img_pre = preprocess_input(img_np)
    AA.append(img_pre)

  K = np.array(AA)
    
  return i, K

In [17]:
seq01 = 0
seq02 = 1001

K = npPath[seq01:seq02]
i, K = dataflow2(K)

index_pre = model2.predict(K).argmax(axis=-1)
index_pre

array([1, 3, 1, ..., 3, 1, 3])

In [ ]:
# ^^^^^^^^^^^^
# colab的ram一直爆炸，我決定重寫。

In [19]:
pre_df = pd.DataFrame({"機器預測":index_pre})

pre_df

,機器預測
0,1
1,3
2,1
3,1
4,3
...,...
996,1
997,3
998,3
999,1


In [20]:
df2 = df2.drop(["Label"], axis=1)

result = pd.concat([df2, pre_df], axis=1) 
result[0:1001]

,ID,機器預測
0,test_00000.png,1.0
1,test_00001.png,3.0
2,test_00002.png,1.0
3,test_00003.png,1.0
4,test_00004.png,3.0
...,...,...
996,test_00996.png,1.0
997,test_00997.png,3.0
998,test_00998.png,3.0
999,test_00999.png,1.0


In [ ]:
# ^^^^^^^^^^^^
# 模型的ram一直掛掉，我覺得應該要寫個新版。